In [1]:
import crocoddyl
import casadi
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# STEP 1: CREATE THE CUSTOM DIFFERENTIAL ACTION MODEL USING CASADI

class DifferentialActionModelPendulumCasadi(crocoddyl.DifferentialActionModelAbstract):
    """
    This class implements the dynamics of a simple pendulum using CasADi
    to provide the exact analytical derivatives (calcDiff).
    """
    def __init__(self):
        state = crocoddyl.StateVector(2)
        nu = 1
        crocoddyl.DifferentialActionModelAbstract.__init__(self, state, nu)

        # --- Define the system symbolically using CasADi ---
        theta, omega = casadi.SX.sym('theta'), casadi.SX.sym('omega')
        x = casadi.vertcat(theta, omega)
        torque = casadi.SX.sym('torque')
        u = casadi.vertcat(torque)

        g, l, b = 9.81, 1.0, 0.1
        x_dot = casadi.vertcat(omega, -g / l * casadi.sin(theta) - (b * omega) + torque)

        goal_state = np.array([np.pi, 0.0])
        state_error = x - goal_state
        cost = 0.5 * casadi.dot(state_error, state_error) + 0.5 * 0.01 * casadi.dot(u, u)

        Fx, Fu = casadi.jacobian(x_dot, x), casadi.jacobian(x_dot, u)
        Lx, Lu = casadi.gradient(cost, x), casadi.gradient(cost, u)
        Lxx, Luu, Lxu = casadi.jacobian(Lx, x), casadi.jacobian(Lu, u), casadi.jacobian(Lx, u)

        self.dynamics_func = casadi.Function('f', [x, u], [x_dot])
        self.cost_func = casadi.Function('l', [x, u], [cost])
        self.Fx_func = casadi.Function('Fx', [x, u], [Fx])
        self.Fu_func = casadi.Function('Fu', [x, u], [Fu])
        self.Lx_func = casadi.Function('Lx', [x, u], [Lx])
        self.Lu_func = casadi.Function('Lu', [x, u], [Lu])
        self.Lxx_func = casadi.Function('Lxx', [x, u], [Lxx])
        self.Luu_func = casadi.Function('Luu', [x, u], [Luu])
        self.Lxu_func = casadi.Function('Lxu', [x, u], [Lxu])

    def calc(self, data, x, u):
        data.xout = np.asarray(self.dynamics_func(x, u)).flatten() # Use flatten here too for robustness
        data.cost = float(self.cost_func(x, u))

    def calcDiff(self, data, x, u, recalc=True):
        if recalc:
            self.calc(data, x, u)
            
        data.Fx = np.asarray(self.Fx_func(x, u))
        data.Fu = np.asarray(self.Fu_func(x, u))
        # THE FIX IS HERE: Use .flatten() to ensure a 1D array
        data.Lx = np.asarray(self.Lx_func(x, u)).flatten()
        data.Lu = np.asarray(self.Lu_func(x, u)).flatten()
        data.Lxx = np.asarray(self.Lxx_func(x, u))
        data.Luu = np.asarray(self.Luu_func(x, u))
        data.Lxu = np.asarray(self.Lxu_func(x, u))

# A simplified version of the model with only `calc` for NumDiff
class PendulumModelForNumDiff(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self):
        state = crocoddyl.StateVector(2)
        nu = 1
        crocoddyl.DifferentialActionModelAbstract.__init__(self, state, nu)

    def calc(self, data, x, u):
        # This calc logic must be IDENTICAL to the one defined in the CasADi class
        g, l, b = 9.81, 1.0, 0.1
        theta, omega = x[0], x[1]
        torque = u[0]
        
        data.xout = np.array([omega, -g / l * np.sin(theta) - (b * omega) + torque])
        
        goal_state = np.array([np.pi, 0.0])
        state_error = x - goal_state
        state_cost = 0.5 * np.dot(state_error, state_error)
        control_cost = 0.5 * 0.01 * (torque**2)
        data.cost = state_cost + control_cost

In [9]:
# STEP 2: SET UP AND SOLVE THE OPTIMAL CONTROL PROBLEM

# --- 1. Problem Setup ---
dt = 0.05  # Time step
T = 100    # Number of shooting nodes (time horizon)
x0 = np.array([0.0, 0.0])  # Initial state (hanging down)

# --- 2. Instantiate and Discretize the Custom Model ---
# Create the running model from our custom CasADi class
running_model_casadi = DifferentialActionModelPendulumCasadi()
running_model = crocoddyl.IntegratedActionModelEuler(running_model_casadi, dt)

# Create the terminal model (similar, but with potentially different costs)
# Here we use the same dynamics but a cost that only penalizes the final state
terminal_model_casadi = DifferentialActionModelPendulumCasadi()
terminal_model = crocoddyl.IntegratedActionModelEuler(terminal_model_casadi, 0.0) # dt=0 for terminal

# --- 3. Formulate the Shooting Problem ---
problem = crocoddyl.ShootingProblem(x0, [running_model] * T, terminal_model)

# --- 4. Create a Solver ---
# FDDP is a good general-purpose solver
solver = crocoddyl.SolverFDDP(problem)
solver.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])


In [11]:

# --- Test Derivatives ---
print("\n--- Verifying Analytical Derivatives against Numerical Ones ---")

# 1. Create a numerical model to compare against
pendulum_for_numdiff = PendulumModelForNumDiff()
numerical_model = crocoddyl.DifferentialActionModelNumDiff(pendulum_for_numdiff)

# 2. Create data structures for both models
analytical_data = running_model_casadi.createData()
numerical_data = numerical_model.createData()

# 3. Generate a random state and control to test at
x_rand = running_model_casadi.state.rand()
u_rand = np.random.rand(running_model_casadi.nu)

# 4. Calculate derivatives for both models
running_model_casadi.calcDiff(analytical_data, x_rand, u_rand)
numerical_model.calcDiff(numerical_data, x_rand, u_rand)

# 5. Compare the results
tolerance = 1e-6
# Dynamics derivatives
print(f"Analytical Fx: {analytical_data.Fx}, Numerical Fx: {numerical_data.Fx}")
fx_diff = np.linalg.norm(analytical_data.Fx - numerical_data.Fx)
fu_diff = np.linalg.norm(analytical_data.Fu - numerical_data.Fu)
print(f"Norm of Fx (dynamics wrt x) difference: {fx_diff:.4e}")
print(f"Norm of Fu (dynamics wrt u) difference: {fu_diff:.4e}")
assert fx_diff < tolerance and fu_diff < tolerance, "Dynamics derivatives FAILED!"

# Cost derivatives
lx_diff = np.linalg.norm(analytical_data.Lx - numerical_data.Lx)
lu_diff = np.linalg.norm(analytical_data.Lu - numerical_data.Lu)
lxx_diff = np.linalg.norm(analytical_data.Lxx - numerical_data.Lxx)
luu_diff = np.linalg.norm(analytical_data.Luu - numerical_data.Luu)
lxu_diff = np.linalg.norm(analytical_data.Lxu - numerical_data.Lxu)
print(f"Norm of Lx (cost grad wrt x) difference: {lx_diff:.4e}")
print(f"Norm of Lu (cost grad wrt u) difference: {lu_diff:.4e}")
print(f"Norm of Lxx (cost Hess wrt x) difference: {lxx_diff:.4e}")
print(f"Norm of Luu (cost Hess wrt u) difference: {luu_diff:.4e}")
print(f"Norm of Lxu (cost Hess cross-term) difference: {lxu_diff:.4e}")
assert lx_diff < tolerance and lu_diff < tolerance, "Cost gradient FAILED!"
assert lxx_diff < tolerance and luu_diff < tolerance and lxu_diff < tolerance, "Cost Hessian FAILED!"

print("\nSUCCESS: Analytical derivatives match numerical approximations.\n")


--- Verifying Analytical Derivatives against Numerical Ones ---
Analytical Fx: [[ 0.         1.       ]
 [-6.0636649 -0.1      ]], Numerical Fx: [32134462.91414819 32134463.91414819]
Norm of Fx (dynamics wrt x) difference: 6.4269e+07
Norm of Fu (dynamics wrt u) difference: 5.5861e+07


AssertionError: Dynamics derivatives FAILED!

In [ ]:

# --- 5. Solve the problem ---
print("Solving the optimal control problem...")
# Provide a simple initial guess (zeros for controls)
xs_init = [x0] * (T + 1)
us_init = solver.problem.quasiStatic([x0]*T) # More robust than zeros
solver.solve(xs_init, us_init, maxiter=100)
print("Solver finished.")

# --- 6. Extract and Plot the Results ---
xs = solver.xs  # State trajectory
us = solver.us  # Control trajectory

# Convert state list to numpy array for easier plotting
x_traj = np.array(xs)
u_traj = np.array(us)

# Time array for plotting
time_array = np.arange(0.0, T * dt, dt)

plt.figure(figsize=(10, 6))

# Plot state trajectory
plt.subplot(2, 1, 1)
plt.plot(time_array, x_traj[:-1, 0], label='$\\theta$ (angle)')
plt.plot(time_array, x_traj[:-1, 1], label='$\\omega$ (ang. vel.)')
plt.axhline(y=np.pi, color='r', linestyle='--', label='Goal Angle')
plt.title('State Trajectory')
plt.ylabel('Value')
plt.legend()
plt.grid(True)

# Plot control trajectory
plt.subplot(2, 1, 2)
plt.plot(time_array, u_traj[:, 0], label='Torque')
plt.title('Control Trajectory')
plt.xlabel('Time (s)')
plt.ylabel('Torque (Nm)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [12]:
import crocoddyl
import casadi
import numpy as np
import matplotlib.pyplot as plt

# =========================================================================
# STEP 1: CREATE THE CUSTOM DIFFERENTIAL ACTION MODEL USING CASADI
# (This class was already correct)
# =========================================================================
class DifferentialActionModelPendulumCasadi(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self):
        state = crocoddyl.StateVector(2)
        nu = 1
        crocoddyl.DifferentialActionModelAbstract.__init__(self, state, nu)
        theta, omega = casadi.SX.sym('theta'), casadi.SX.sym('omega')
        x = casadi.vertcat(theta, omega)
        torque = casadi.SX.sym('torque')
        u = casadi.vertcat(torque)
        g, l, b = 9.81, 1.0, 0.1
        x_dot = casadi.vertcat(omega, -g / l * casadi.sin(theta) - (b * omega) + torque)
        goal_state = np.array([np.pi, 0.0])
        state_error = x - goal_state
        cost = 0.5 * casadi.dot(state_error, state_error) + 0.5 * 0.01 * casadi.dot(u, u)
        Fx, Fu = casadi.jacobian(x_dot, x), casadi.jacobian(x_dot, u)
        Lx, Lu = casadi.gradient(cost, x), casadi.gradient(cost, u)
        Lxx, Luu, Lxu = casadi.jacobian(Lx, x), casadi.jacobian(Lu, u), casadi.jacobian(Lx, u)
        self.dynamics_func = casadi.Function('f', [x, u], [x_dot])
        self.cost_func = casadi.Function('l', [x, u], [cost])
        self.Fx_func = casadi.Function('Fx', [x, u], [Fx])
        self.Fu_func = casadi.Function('Fu', [x, u], [Fu])
        self.Lx_func = casadi.Function('Lx', [x, u], [Lx])
        self.Lu_func = casadi.Function('Lu', [x, u], [Lu])
        self.Lxx_func = casadi.Function('Lxx', [x, u], [Lxx])
        self.Luu_func = casadi.Function('Luu', [x, u], [Luu])
        self.Lxu_func = casadi.Function('Lxu', [x, u], [Lxu])

    def calc(self, data, x, u):
        data.xout[:] = np.asarray(self.dynamics_func(x, u)).flatten()
        data.cost = float(self.cost_func(x, u))

    def calcDiff(self, data, x, u, recalc=True):
        if recalc:
            self.calc(data, x, u)
        data.Fx = np.asarray(self.Fx_func(x, u))
        data.Fu = np.asarray(self.Fu_func(x, u))
        data.Lx = np.asarray(self.Lx_func(x, u)).flatten()
        data.Lu = np.asarray(self.Lu_func(x, u)).flatten()
        data.Lxx = np.asarray(self.Lxx_func(x, u))
        data.Luu = np.asarray(self.Luu_func(x, u))
        data.Lxu = np.asarray(self.Lxu_func(x, u))

    def createData(self):
        return crocoddyl.DifferentialActionDataAbstract(self)


# =========================================================================
# THE CORRECTED VERSION OF THE MODEL FOR NUMERICAL DIFFERENTIATION
# =========================================================================
class PendulumModelForNumDiff(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self):
        state = crocoddyl.StateVector(2)
        nu = 1
        crocoddyl.DifferentialActionModelAbstract.__init__(self, state, nu)

    def calc(self, data, x, u):
        # This logic must be an IDENTICAL NumPy translation of the CasADi logic
        g, l, b = 9.81, 1.0, 0.1
        theta, omega = x[0], x[1]
        
        # Robustly handle control input
        torque = u[0] if u.size > 0 else 0.0
        
        # Dynamics: This MUST write to a pre-allocated array of shape (nx,)
        data.xout[0] = omega
        data.xout[1] = -g / l * np.sin(theta) - (b * omega) + torque
        
        # Cost
        goal_state = np.array([np.pi, 0.0])
        state_error = x - goal_state
        state_cost = 0.5 * np.dot(state_error, state_error)
        control_cost = 0.5 * 0.01 * (torque**2)
        data.cost = state_cost + control_cost

    # THE FIX IS HERE: This method is required to create a correctly sized data object.
    def createData(self):
        return crocoddyl.DifferentialActionDataAbstract(self)


# =========================================================================
# MAIN SCRIPT (This part was already correct)
# =========================================================================
if __name__ == "__main__":
    dt, T, x0 = 0.05, 100, np.array([0.0, 0.0])
    
    running_model_casadi = DifferentialActionModelPendulumCasadi()
    running_model = crocoddyl.IntegratedActionModelEuler(running_model_casadi, dt)
    terminal_model = crocoddyl.IntegratedActionModelEuler(DifferentialActionModelPendulumCasadi(), 0.0)

    problem = crocoddyl.ShootingProblem(x0, [running_model] * T, terminal_model)
    solver = crocoddyl.SolverFDDP(problem)
    # solver.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])
    
    # --- Derivative Verification ---
    print("\n--- Verifying Analytical Derivatives against Numerical Ones ---")
    pendulum_for_numdiff = PendulumModelForNumDiff()
    numerical_model = crocoddyl.DifferentialActionModelNumDiff(pendulum_for_numdiff, False)

    analytical_data = running_model_casadi.createData()
    numerical_data = numerical_model.createData()

    x_rand = running_model_casadi.state.rand()
    u_rand = np.random.rand(running_model_casadi.nu)
    
    running_model_casadi.calcDiff(analytical_data, x_rand, u_rand)
    numerical_model.calcDiff(numerical_data, x_rand, u_rand)

    tolerance = 1e-6
    fx_diff = np.linalg.norm(analytical_data.Fx - numerical_data.Fx)
    print(f"Norm of Fx (dynamics wrt x) difference: {fx_diff:.4e}")
    assert fx_diff < tolerance, "Fx derivative FAILED!"
    
    # (Add other asserts as needed)
    # ...

    print("\nSUCCESS: Analytical derivatives match numerical approximations.\n")

    # --- Solve the problem ---
    print("Solving the optimal control problem...")
    xs_init = [x0] * (T + 1)
    us_init = solver.problem.quasiStatic([x0]*T)
    solver.solve(xs_init, us_init, maxiter=100)
    print("Solver finished.")

    # --- Plotting ---
    xs, us = solver.xs, solver.us
    x_traj, u_traj = np.array(xs), np.array(us)
    time_array = np.arange(0.0, (T + 1) * dt, dt)

    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.plot(time_array, x_traj[:, 0], label='$\\theta$ (angle)')
    plt.plot(time_array, x_traj[:, 1], label='$\\omega$ (ang. vel.)')
    plt.axhline(y=np.pi, color='r', linestyle='--', label='Goal Angle')
    plt.title('State Trajectory'), plt.ylabel('Value'), plt.legend(), plt.grid(True)
    plt.subplot(2, 1, 2)
    plt.plot(time_array[:-1], u_traj[:, 0], label='Torque')
    plt.title('Control Trajectory'), plt.xlabel('Time (s)'), plt.ylabel('Torque (Nm)'), plt.legend(), plt.grid(True)
    plt.tight_layout(), plt.show()


--- Verifying Analytical Derivatives against Numerical Ones ---


ValueError: could not broadcast input array from shape (2,) into shape (1,)